In [41]:
from Naive_bayes import NaiveBayes
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance 
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
import time
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numba import jit
import numpy as np
%matplotlib inline

In [42]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )
y = pd.read_csv("results/fuzzy_crisp_partition_572953998.csv",sep="\n", header=None)
y = y.values
y  = [y[i][0] for i in range(len(y))]
y = np.array(y)

In [43]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [44]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
  
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   

In [45]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

In [46]:
kf = KFold(n_splits=10,shuffle=True)

In [47]:

list_acc_n = []
count = 0


for i in trange(0, 30):
    acc_fold = []
    for train_index, test_index in kf.split(np.array((mfeat))):
       
        X_trains = []
        y_train_views = []
        X_tests = []
    
        X_train_view1, X_test_view1 = mfeat[train_index], mfeat[test_index]
        y_train_view1, y_test_view1 = y[train_index], y[test_index]
      
       
       
        X_train_view2, X_test_view2 = mfeatFou[train_index], mfeatFou[test_index]
        y_train_view2, y_test_view2 = y[train_index], y[test_index]
       
   

        X_train_view3, X_test_view3 = mfeatKa[train_index], mfeatKa[test_index]
        y_train_view3, y_test_view3 = y[train_index], y[test_index]
        
  
        
        ##Values   
        X_trains = [X_train_view1, X_train_view2, X_train_view3]
        y_train_views = [y_train_view1, y_train_view2, y_train_view3]
        X_tests = [X_test_view1, X_test_view2, X_test_view3]
        nb = NaiveBayes()
        
        nb.fit(X_trains, y_train_views)
        
        y_pred = nb.predict(X_tests=X_tests);
        
        acc_fold = accuracy_score(y_test_view1, y_pred)
        
    list_acc_n.append(np.mean(acc_fold))


100%|██████████| 30/30 [38:30<00:00, 80.64s/it]


In [48]:
acc_naive = pd.DataFrame(list_acc_n, columns=['acc'])
acc_naive_csv = acc_naive.to_csv()
arq = open("results/accuracy_naive", "w")
arq.write(acc_naive_csv)
arq.close()

In [50]:
 np.mean(list_acc_n)

0.6664999999999998

## Validação do KNN

In [28]:
from knn import KNN

In [29]:
k = KNN()

In [51]:

X_train_mfeat, X_test_mfeat, y_train_mfeat, y_test_mfeat =  train_test_split(mfeat,y, test_size=0.10, random_state=42)
X_train_mfeatFou, X_test_mfeatFou, y_train_mfeatFou, y_test_mfeatFou =  train_test_split(mfeatFou,y, test_size=0.10, random_state=42)
X_train_mfeatKa, X_test_mfeatKa, y_train_mfeatKa, y_test_mfeatKa =  train_test_split(mfeatKa,y, test_size=0.10, random_state=42)

In [24]:
trains = [X_train_mfeat, X_train_mfeatFou, X_train_mfeatKa]
y_trains = [y_train_mfeat,y_train_mfeatFou, y_train_mfeatKa]
tests = [X_test_mfeat, X_test_mfeatFou, X_test_mfeatKa]

In [31]:
ks = [3,5,7, 11, 13]
for k in ks:
    model_knn = KNN()
    model_knn.fit(X_trains=trains, y_trains=y_trains, k=k)
    y =model_knn.predict(X_tests=tests)   
    print("K: ", k, " ACC: ", accuracy_score(y_test_mfeatFou, y))


K:  3  ACC:  0.75
K:  5  ACC:  0.75
K:  7  ACC:  0.76
K:  11  ACC:  0.745
K:  13  ACC:  0.74


In [52]:
kf = KFold(n_splits=10,shuffle=True)

In [54]:
list_acc_knn = []


for i in trange(0, 30):
    acc_fold = []
    for train_index, test_index in kf.split(np.array(X_train_mfeat)):
       
        X_trains = []
        y_train_views = []
        X_tests = []
    
        X_train_view1, X_test_view1 = mfeat[train_index], mfeat[test_index]
        y_train_view1, y_test_view1 = y[train_index], y[test_index]
      
       
       
        X_train_view2, X_test_view2 = mfeatFou[train_index], mfeatFou[test_index]
        y_train_view2, y_test_view2 = y[train_index], y[test_index]
       
   

        X_train_view3, X_test_view3 = mfeatKa[train_index], mfeatKa[test_index]
        y_train_view3, y_test_view3 = y[train_index], y[test_index]
        
  
        
        ##Values   
        X_trains = [X_train_view1, X_train_view2, X_train_view3]
        y_trains = [y_train_view1, y_train_view2, y_train_view3]
        X_tests = [X_test_view1, X_test_view2, X_test_view3]
        k = KNN()
        
        k.fit(X_trains=X_trains, y_trains=y_trains, k=7)
        
        y_pred = k.predict(X_tests=X_tests);
        
        acc_fold = accuracy_score(y_test_view1, y_pred)
        
    list_acc_knn.append(np.mean(acc_fold))


100%|██████████| 30/30 [1:24:58<00:00, 169.73s/it]


In [55]:
np.round(np.mean(list_acc_knn), 3)

0.744

In [56]:
acc_knn = pd.DataFrame(list_acc_knn, columns=['acc'])
acc_knn_csv = acc_knn.to_csv()
arq = open("results/accuracy_knn", "w")
arq.write(acc_knn_csv)
arq.close()

In [57]:
acc_naive = pd.read_csv("results/accuracy_naive")

In [58]:
acc_naive = acc_naive['acc'].values

In [59]:
from scipy.stats import wilcoxon

In [60]:
test = wilcoxon(acc_naive, list_acc_knn)

In [61]:
test.pvalue < 0.05

True